# Yelp busniess ratings prediction

This and subsequent notebooks predict the ratings of the business based on the reviews using the dataset provided in [RecSys2013: Yelp Business Rating Prediction](https://www.kaggle.com/c/yelp-recsys-2013/overview) 

ach observation in this dataset is a review of a particular business by a particular user.

The "stars" column is the number of stars (1 through 5) assigned by the reviewer to the business. (Higher stars is better.) 

There is 'votes' colum that has three subcategories as "cool", "useful" and "funny". These are ratings of the 'review' itself from other Yelp users,  and not the rating of the business. column is the number of "cool" votes this review received from other Yelp users. 


In [19]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

%matplotlib inline
fig_params={'legend.fontsize':18,
          'axes.labelsize': 18,
          'axes.titlesize':20,
          'xtick.labelsize':16,
          'ytick.labelsize':16,             
          'axes.facecolor':'#D9DDD1'}
plt.rcParams.update(fig_params)

In [2]:
df_review=pd.read_json('./data/yelp-recsys-2013/yelp_training_set/yelp_training_set_review.json',lines=True)

In [3]:
df_review.head()

,votes,user_id,review_id,stars,date,text,type,business_id
0,"{'funny': 0, 'useful': 5, 'cool': 2}",rLtl8ZkDX5vH5nAx9C3q5Q,fWKvX83p0-ka4JS3dc6E5A,5,2011-01-26,My wife took me here on my birthday for breakf...,review,9yKzy9PApeiPPOUJEtnvkg
1,"{'funny': 0, 'useful': 0, 'cool': 0}",0a2KyEL0d3Yb1V6aivbIuQ,IjZ33sJrzXqU-0X6U8NwyA,5,2011-07-27,I have no idea why some people give bad review...,review,ZRJwVLyzEJq1VAihDhYiow
2,"{'funny': 0, 'useful': 1, 'cool': 0}",0hT2KtfLiobPvh6cDC8JQg,IESLBzqUCLdSzSqm0eCSxQ,4,2012-06-14,love the gyro plate. Rice is so good and I als...,review,6oRAC4uyJCsJl1X0WZpVSA
3,"{'funny': 0, 'useful': 2, 'cool': 1}",uZetl9T0NcROGOyFfughhg,G-WvGaISbqqaMHlNnByodA,5,2010-05-27,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,_1QQZuf4zZOyFCvXc0o6Vg
4,"{'funny': 0, 'useful': 0, 'cool': 0}",vYmM4KTsC8ZfQBg-j5MWkw,1uJFq2r5QfJG_6ExMRCaGw,5,2012-01-05,General Manager Scott Petello is a good egg!!!...,review,6ozycU1RpktNG2-1BroVtw


In [4]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229907 entries, 0 to 229906
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   votes        229907 non-null  object        
 1   user_id      229907 non-null  object        
 2   review_id    229907 non-null  object        
 3   stars        229907 non-null  int64         
 4   date         229907 non-null  datetime64[ns]
 5   text         229907 non-null  object        
 6   type         229907 non-null  object        
 7   business_id  229907 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 14.0+ MB


In [5]:
df_review[['funny','useful','cool']]=df_review['votes'].apply(pd.Series)

In [6]:
df_review.drop(['votes'],axis=1,inplace=True)

In [7]:
df_review.head()

,user_id,review_id,stars,date,text,type,business_id,funny,useful,cool
0,rLtl8ZkDX5vH5nAx9C3q5Q,fWKvX83p0-ka4JS3dc6E5A,5,2011-01-26,My wife took me here on my birthday for breakf...,review,9yKzy9PApeiPPOUJEtnvkg,0,5,2
1,0a2KyEL0d3Yb1V6aivbIuQ,IjZ33sJrzXqU-0X6U8NwyA,5,2011-07-27,I have no idea why some people give bad review...,review,ZRJwVLyzEJq1VAihDhYiow,0,0,0
2,0hT2KtfLiobPvh6cDC8JQg,IESLBzqUCLdSzSqm0eCSxQ,4,2012-06-14,love the gyro plate. Rice is so good and I als...,review,6oRAC4uyJCsJl1X0WZpVSA,0,1,0
3,uZetl9T0NcROGOyFfughhg,G-WvGaISbqqaMHlNnByodA,5,2010-05-27,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,_1QQZuf4zZOyFCvXc0o6Vg,0,2,1
4,vYmM4KTsC8ZfQBg-j5MWkw,1uJFq2r5QfJG_6ExMRCaGw,5,2012-01-05,General Manager Scott Petello is a good egg!!!...,review,6ozycU1RpktNG2-1BroVtw,0,0,0


In [8]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229907 entries, 0 to 229906
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   user_id      229907 non-null  object        
 1   review_id    229907 non-null  object        
 2   stars        229907 non-null  int64         
 3   date         229907 non-null  datetime64[ns]
 4   text         229907 non-null  object        
 5   type         229907 non-null  object        
 6   business_id  229907 non-null  object        
 7   funny        229907 non-null  int64         
 8   useful       229907 non-null  int64         
 9   cool         229907 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 17.5+ MB


In [9]:
y=df_review['stars']

In [10]:
X=df_review.drop(['stars'],axis=1)

In [11]:
y

0         5
1         5
2         4
3         5
4         5
         ..
229902    2
229903    1
229904    4
229905    3
229906    5
Name: stars, Length: 229907, dtype: int64

In [12]:
X_temp,X_test,y_temp,y_test=train_test_split(X,y,test_size=0.2,random_state=40)
X_train,X_cv,y_train,y_cv=train_test_split(X_temp,y_temp,test_size=0.2,random_state=101)

In [13]:
df_train=pd.concat([X_train,y_train],axis=1)
df_val=pd.concat([X_cv,y_cv],axis=1)
df_test=pd.concat([X_test,y_test],axis=1)

In [14]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147140 entries, 106415 to 216615
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   user_id      147140 non-null  object        
 1   review_id    147140 non-null  object        
 2   date         147140 non-null  datetime64[ns]
 3   text         147140 non-null  object        
 4   type         147140 non-null  object        
 5   business_id  147140 non-null  object        
 6   funny        147140 non-null  int64         
 7   useful       147140 non-null  int64         
 8   cool         147140 non-null  int64         
 9   stars        147140 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 12.3+ MB


In [20]:
pickle.dump(df_train,open('./data/train.pkl','wb'))
pickle.dump(df_val,open('./data/val.pkl','wb'))
pickle.dump(df_test,open('./data/test.pkl','wb'))